In [2]:
import pandas as pd

In [3]:
data=pd.read_csv("D:/ML/House_Price_Prediction/notebooks/data/Housing.csv")

In [4]:
data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [5]:
data.columns

Index(['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'hotwaterheating', 'airconditioning',
       'parking', 'prefarea', 'furnishingstatus'],
      dtype='object')

In [6]:
data['price'].min()

1750000

In [7]:
(671879.5914634146/4766729.247706422)*100 #  percentage error 

14.095190990482601

In [8]:
X=data.drop(labels=['price'],axis=1)

In [9]:
y=data['price']

In [10]:
cat_column=data.columns[data.dtypes=='O']

In [11]:
X=data.drop(labels=['price'],axis=True)

In [12]:
y=data['price']

In [13]:
num_column=X.columns[X.dtypes!='O']

In [14]:
cat_column

Index(['mainroad', 'guestroom', 'basement', 'hotwaterheating',
       'airconditioning', 'prefarea', 'furnishingstatus'],
      dtype='object')

In [15]:
num_column

Index(['area', 'bedrooms', 'bathrooms', 'stories', 'parking'], dtype='object')

In [16]:
mainroad=['no','yes']
guestroom=['no','yes']
basement=['no','yes']
hotwaterheating=['no','yes']
airconditioning=['no','yes']
prefarea=['no','yes']
furnishingstatus=['unfurnished','semi-furnished','furnished']

In [17]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [18]:
num_pipeline=Pipeline(
    
    steps=[
        ('imputer',SimpleImputer()),
        ('scaling',StandardScaler())
    ]
)

In [19]:
cat_pipeline=Pipeline(
    
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('encoding',OrdinalEncoder(categories=[mainroad,basement,guestroom,airconditioning,
                                               hotwaterheating,prefarea,furnishingstatus]))
    ]
)

In [20]:
preprocessor=ColumnTransformer(
[
    ('num',num_pipeline,num_column),
    ('cat',cat_pipeline,cat_column)
]

)

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.15,random_state=21)

In [22]:
preprocessor.fit_transform(X_train)

array([[-0.99019291, -1.31006766, -0.56710359, ...,  0.        ,
         0.        ,  0.        ],
       [-0.42170164,  1.40385326, -0.56710359, ...,  1.        ,
         0.        ,  2.        ],
       [ 0.38909738,  0.0468928 ,  1.40709914, ...,  1.        ,
         0.        ,  2.        ],
       ...,
       [ 0.22134586, -1.31006766, -0.56710359, ...,  0.        ,
         0.        ,  1.        ],
       [-0.40306259,  0.0468928 ,  1.40709914, ...,  0.        ,
         0.        ,  0.        ],
       [-1.00883197,  0.0468928 , -0.56710359, ...,  1.        ,
         0.        ,  1.        ]])

In [23]:
preprocessor.transform(X_test)

array([[ 0.15610916,  0.0468928 ,  1.40709914,  0.20844122,  0.3627041 ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ],
       [ 0.41705597, -1.31006766, -0.56710359, -0.94046692,  0.3627041 ,
         1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  1.        ],
       [ 0.85507383,  0.0468928 ,  1.40709914,  2.50625749,  1.51292325,
         1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  2.        ],
       [ 0.66868325,  1.40385326,  1.40709914,  0.20844122,  0.3627041 ,
         1.        ,  1.        ,  1.        ,  0.        ,  1.        ,
         1.        ,  0.        ],
       [ 0.38909738,  1.40385326,  1.40709914,  2.50625749,  0.3627041 ,
         1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  1.        ],
       [-1.21852137,  0.0468928 , -0.56710359,  0.20844122, -0.78751506,
         1.        ,  

In [24]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [25]:
X_train.head()

,num__area,num__bedrooms,num__bathrooms,num__stories,num__parking,cat__mainroad,cat__guestroom,cat__basement,cat__hotwaterheating,cat__airconditioning,cat__prefarea,cat__furnishingstatus
0,-0.990193,-1.310068,-0.567104,-0.940467,-0.787515,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.421702,1.403853,-0.567104,0.208441,-0.787515,1.0,0.0,1.0,0.0,1.0,0.0,2.0
2,0.389097,0.046893,1.407099,2.506257,0.362704,1.0,0.0,0.0,0.0,1.0,0.0,2.0
3,-0.715267,1.403853,-0.567104,0.208441,2.663142,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.743225,0.046893,-0.567104,0.208441,-0.787515,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [26]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [27]:
def model_evaluate(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rsme=np.sqrt(mean_squared_error(true,predicted))
    r_square=r2_score(true,predicted)

    return mae,mse,rsme,r_square    

In [28]:
models={
    'LinearRegression': LinearRegression(),
    'ridge':Ridge(),
    'Lasso':Lasso(),
    'ElasticNet':ElasticNet(),
    'RandomForestRegressor':RandomForestRegressor()
}

In [29]:
model_list=[]
r2_list=[]

In [30]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    
    #model prediction
    y_pred=model.predict(X_test)
    
    mae,mse,rsme,r_square=model_evaluate(y_test,y_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('model training performance')
    print("mean_absolute_error : ",mae)
    print('mean_squared_error : ',mse)
    print("rmse : ",rsme)
    print('r_square : ',r_square*100)
    
    r2_list.append(r_square)

    print('-'*30)
    print('\n')

LinearRegression
model training performance
mean_absolute_error :  817108.8469100344
mean_squared_error :  1157963510347.1384
rmse :  1076087.1295332634
r_square :  61.85647822645011
------------------------------


ridge
model training performance
mean_absolute_error :  816023.8730322747
mean_squared_error :  1158683457643.813
rmse :  1076421.5984658673
r_square :  61.832763035823525
------------------------------


Lasso
model training performance
mean_absolute_error :  817107.5162344788
mean_squared_error :  1157963348831.3364
rmse :  1076087.0544855264
r_square :  61.85648354680904
------------------------------


ElasticNet
model training performance
mean_absolute_error :  798286.7668653695
mean_squared_error :  1201149935989.446
rmse :  1095969.8608946535
r_square :  60.433909767176864
------------------------------


RandomForestRegressor
model training performance
mean_absolute_error :  685962.4105691058
mean_squared_error :  861447410945.0583
rmse :  928141.9131496316
r_square

In [31]:
r2_list

[0.6185647822645011,
 0.6183276303582352,
 0.6185648354680904,
 0.6043390976717686,
 0.7162377071251533]

In [32]:
from src.HousePricePrediction.logger import logging

In [33]:
logging.info("hey")

In [34]:
linear=LinearRegression()

In [35]:
from sklearn.ensemble import RandomForestRegressor

In [36]:
rfg=RandomForestRegressor()

In [37]:
rfg.fit(X_train,y_train)

RandomForestRegressor()

In [38]:
y_pred1=rfg.predict(X_test)

In [39]:
r2_score(y_test,y_pred1)

0.700821709702993

In [40]:
mean_squared_error(y_test,y_pred1)

908247396002.6677

In [41]:
mean_absolute_error(y_test,y_pred1)

697697.5548780488

In [75]:
from sklearn.model_selection import GridSearchCV

In [44]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

grid_search = GridSearchCV(estimator=rfg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)


KeyboardInterrupt: 

In [40]:
y_pred2=grid_search.predict(X_test)

In [41]:
r2_score(y_test,y_pred2)*100

69.74510010282633

In [42]:
from pandas_profiling import ProfileReport

d:\ML\House_Price_Prediction\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\nikam\AppData\Local\Temp\ipykernel_5164\2274191625.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [43]:
from pandas_profiling import ProfileReport


profile = ProfileReport(data,config_file='')

profile.to_file("house_price_report.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 62.09it/s]
